In [1]:
import transformers
import shap
import numpy
import matplotlib.pylab as plt
import pandas as pd
import numpy as np
import scipy
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

# Opis rozwiązania
Nasze rozwiązanie składa się z 3 modułów. Dwa z nich to modele, które pozwalają klasyfikować tekst. A trzeci służy do objaśniania predykcji modelu. Można je łączyć na kilka sposobów, które znajdą zastosowanie w wielu produktach takich jak:
1. Analiza sentymentu po stronie użytkownika. Wpisując komentarz otrzymujemy informację czy nie jest on nacechowany negatywnie (toksyczność, groźba, dyskryminacja grup). Jeżeli jest, to nasz moduł objaśniania wskaże dla każdej z negatywnych cech miejsce jej występowania w komentarzu, które użytkownik może.
2. Automatyczne zgłaszanie negatywnych komentarzy. Po wykryciu negatywnego sentymentu moderator otrzymuje komentarz z oznaczonym negatywnym fragmentem, co przyspiesza i ułatwia moderację.

# Moduł objaśniania modeli
Jest to nasz najważniejszy moduł, korzystając z biblioteki SHAP można objaśniać dowolne modele, także te tekstowe. Poniżej znajduje się przykład z biblioteki shap, w kolejnych rozdziałach pokażemy zastosowanie tego modułu we współpracy z naszymi modelami.
![](./img/shap.png)

Tutaj dla każdego fragmentu tekstu, oznaczone mamy jaki miał wpływ na wyjście modelu. Dzięki temu jesteśmy w stanie dla wybranego negatywnego zachowania określić miejsce jego występowania w tekście.

# Moduł badania sentymentu
Moduł ten odpowiada za badanie sentymenu, sklasyfikuje dowolny tekst do jednej z 3 klas: 
- Negative
- Neutral
- Positive


In [4]:
sentiment_pipe = pipeline("text-classification", model="cardiffnlp/twitter-roberta-base-sentiment")

def translate(label):
    t = {"LABEL_0": "Negative",
     "LABEL_1": "Neutral",
     "LABEL_2": "Positive"}
    return t[label]

In [8]:
def predict_sentiment(data):
    label = translate(pipe(data)[0]["label"])
    return {"label": label, "score": pipe(data)[0]["score"]}

In [34]:
text = "you dumb clown"
predict_sentiment(text)

{'label': 'Negative', 'score': 0.9534616470336914}

Mając negatywny wynik możemy z pomocą `Modułu objaśniania modeli` znaleźć co sprawiło, że model oznaczył ten tekst jako `Negative`.

In [35]:
explainer = shap.Explainer(pipe) # Moduł objaśniania
shap_values = explainer([text])
label = pipe(text)[0]["label"]
shap.plots.text(shap_values[:,:, label]) # Wybieramy klasę objaśnianą

W skali od niebieskiego do czerwonego mamy pokazane, które fragmenty tekstu zadecyowały o wyniku modelu. Tutaj słowa `dumb` oraz `clown` zadecydowały o  klasie `Negative`.
Jeżeli zmienimy tekst, sprawimy, że będzie bardziej pozytywny.

In [36]:
text = "you dumb clown, but I still like you"

In [37]:
print(predict_sentiment(text))
shap_values = explainer([text])
label = pipe(text)[0]["label"]
shap.plots.text(shap_values[:,:, label]) # Wybieramy klasę objaśnianą

{'label': 'Negative', 'score': 0.5696905851364136}


Sentyment nadal jest `Negative`, ale widzimy, że moduł objaśniający pokazuje nam, które słowa zmniejszyły prawdopodobieństwo tej klasy.

# Moduł klasyfikacji negatywnego tekstu
Dla dowolnego tekstu oprócz tego, że tekst jest negatywny możemy określić dlaczego jest negatywny tj. wskazać prawdopodobieństwo należenia do każdej z klas:
- toxic
- severe_toxic
- obscene
- threat
- insult
- identity_hate

Wracając do poprzedniego przykładu, obliczymy te prawdopodobieństwa.

In [44]:
clf_tokenizer = AutoTokenizer.from_pretrained("unitary/toxic-bert")
clf_model = AutoModelForSequenceClassification.from_pretrained("unitary/toxic-bert")
clf_pred = transformers.pipeline("text-classification", model=clf_model, tokenizer=clf_tokenizer, return_all_scores=True)

In [45]:
text = "you dumb clown"
clf_pred(text)

[[{'label': 'toxic', 'score': 0.7617954015731812},
  {'label': 'severe_toxic', 'score': 0.00035054481122642756},
  {'label': 'obscene', 'score': 0.014293862506747246},
  {'label': 'threat', 'score': 1.986711322388146e-05},
  {'label': 'insult', 'score': 0.22305439412593842},
  {'label': 'identity_hate', 'score': 0.0004858026804868132}]]

Najbardziej prawdopodobne jest, że tekst ten zawiera klasy: `toxic` oraz `insult`. Poniżej objaśniłem dlaczgo model z dużym prawdopodobieństwem przypisał klasę `toxic`:

In [52]:
clf_explainer = shap.Explainer(clf_pred)
shapley_values = explainer(np.array([text]))
shap.plots.text(shapley_values[:, :, 'toxic']) # Klasa toxic

Oraz klasę `insult`:

In [53]:
shap.plots.text(shapley_values[:, :, 'insult']) # Klasa insult